RTS from text files

In [1]:
import pandas as pd
from databaker.framework import *
from pathlib import Path
# from tqdm import tqdm

In [2]:
def user_perc(x):

        if x.strip(' ') == '':
            return 'NA'
        else:
            return x

In [3]:
def user_perc2(x):

        if x.strip(' ') == 'A':
            return 'Asia and Oceania'
        elif x.strip(' ') == 'B':  
            return 'Eastern Europe'
        elif x.strip(' ') == 'C':  
            return 'European Union'
        elif x.strip(' ') == 'D':  
            return 'Latin America and the Caribbean'
        elif x.strip(' ') == 'E':  
            return 'Low Value Trade (Non-EU)'
        elif x.strip(' ') == 'F':  
            return 'Middle East and North Africa'
        elif x.strip(' ') == 'G':  
            return 'North America'
        elif x.strip(' ') == 'H':  
            return 'Sub-Saharan Africa'
        elif x.strip(' ') == 'I':  
            return 'Western Europe'
        elif x.strip(' ') == 'J':  
            return 'Ships and Stores(Non-EU)'
        else:  
            return 'NA'

In [4]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from io import BytesIO

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

google_id = [f'https://drive.google.com/uc?export=download&id=1gJDwtdg1NJ9ybtlHmgsIklkhThaBqVJX',
            f'https://drive.google.com/uc?export=download&id=1cvel3cvNAwbdN6l1nVfBOwNyfj0Ken2g',
            f'https://drive.google.com/uc?export=download&id=1945hQ8L9-NwZMX6B4zZcshJBORcXVZuy',
            f'https://drive.google.com/uc?export=download&id=1y8SRiOxyDlMJmuWMmV005VoxUpk0na1l',
            f'https://drive.google.com/uc?export=download&id=1ZHJ6lRUHonVLlALeplnMfvdEhgVedtp5',
            f'https://drive.google.com/uc?export=download&id=1y7aA8oGo__4YErK09KHZGaRGs79_lauE',
            f'https://drive.google.com/uc?export=download&id=1FfApFvt72ogPkhhJfxevR3Fd_otGS-Es',
            f'https://drive.google.com/uc?export=download&id=1mZmI8nprCCPwl1kaBhT3dCCC-FXk0CMo']

Final_table = pd.DataFrame()
for g_id in google_id:
    
    input_file = BytesIO(session.get(g_id).content)
    new_table = pd.read_csv(input_file, delim_whitespace= True, header = None)
    new_table_data = new_table[new_table[3].isnull() == False]
    new_table = new_table[new_table[3].isnull() == True]
    new_table_data[1] =new_table_data[2]
    new_table_data[2] =new_table_data[3]
    new_table = pd.concat([new_table, new_table_data])
    new_table.drop([3], axis =1, inplace =True)
    new_table['Period'] ="quarter/"+ new_table[0].astype(str).str[2:6]+'-'+new_table[0].astype(str).str[0:2].str[::-1]
    new_table['Flow'] = new_table[0].astype(str).str[6:7]
    new_table['HMRC Reporter Region'] = new_table[0].astype(str).str[7:9]
    new_table['Geography'] = new_table[0].astype(str).str[9:10]
    new_table['Codeseq'] = new_table[0].astype(str).str[10:13]
    new_table['HMRC Partner Geography'] = new_table[0].astype(str).str[13:15]
    new_table['SITC Section'] = new_table[0].astype(str).str[15:16]
    new_table['SITC Division'] = new_table[0].astype(str).str[16:18]
    new_table.rename(index = str, columns = {2:'Mass'}, inplace = True)
    new_table.rename(index = str, columns = {1:'Value'}, inplace = True)
    new_table['Unit'] = '£ (thousands)'
    new_table['Measure Type'] = 'GBP Total'   
    new_table['Geography'] = new_table.apply(lambda row: user_perc2(row['Geography']), axis = 1)
    new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('E', 'Exports'))
    new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('I', 'Imports'))    
    new_table['HMRC Partner Geography'] = new_table.apply(lambda row: user_perc(row['HMRC Partner Geography']), axis = 1)
    new_table['SITC Section'] = new_table.apply(lambda row: user_perc(row['SITC Section']), axis = 1)
    new_table['SITC Division'] = new_table.apply(lambda row: user_perc(row['SITC Division']), axis = 1)
    new_table['SITC 4'] = new_table['SITC Division']  
    new_table['Value'] = new_table['Value'].astype('int', copy=False)
    new_table = new_table[new_table['Value'] != 0 ]
    new_table = new_table[new_table['SITC 4'] != 'NA' ]
    new_table['SITC 4'] = new_table['SITC 4'].map(lambda cell: cell.replace('98', '098'))
    new_table =new_table[['Period','HMRC Reporter Region','HMRC Partner Geography','SITC 4','Flow','Measure Type','Value','Unit']]
    
    destinationFolder = Path('out')
    destinationFolder.mkdir(exist_ok=True, parents=True)
    Final_table = pd.concat([Final_table, new_table])
# Final_table.to_csv(destinationFolder / ('rts_Value_tidydata.csv'), index = False) 
# Final_table.sample(n=10000, random_state=145).to_csv(destinationFolder / ('rts_Value_tidydata.csv'), index = False)

In [5]:
import numpy as np
sliceSize = 10000
for i in np.arange(len(Final_table)//sliceSize):
    destFile = destinationFolder / f'rts_Value_{i:04}.csv'
    Final_table.iloc[i*sliceSize:i*sliceSize+sliceSize-1].to_csv(destFile, index=False)